# Lektion 13

In [ ]:
from sympy import *
init_printing()
from IPython.display import display

## Besselfunktionen

In [ ]:
x = Symbol('x')
y = Function('y')
dgl = Eq(y(x).diff(x, 2), -1/x*y(x).diff(x) + 1/x**2*y(x) +4*y(x))
dgl

In [ ]:
#dsolve(dgl)   # NotImplementedError

In [ ]:
#N = 8
N=18

In [ ]:
a = [Symbol('a'+str(j)) for j in range(N)]

In [ ]:
n = Symbol('n')

In [ ]:
ys = sum([a[j]*x**j for j in range(N)]) 
ys 

In [ ]:
gl = dgl.subs(y(x), ys).doit()
gl

In [ ]:
p1 = (gl.lhs - gl.rhs).expand().powsimp()
p1

In [ ]:
p1.coeff(x**(-2))

In [ ]:
p1.coeff(x**(-1))

In [ ]:
p1.coeff(x, 0)

In [ ]:
gls = []
for j in range(N+1):
    glg = Eq(p1.coeff(x, j-2), 0)
    if glg != True:
        gls.append(glg)
gls

In [ ]:
#solve(dgs) NotImplementedError

In [ ]:
Lsg = solve(gls[:-1])
Lsg

Die ungeraden a werden rückwärts gelöst.  Das ist verwirrend.

In [ ]:
var = a.copy()   # böse Falle
del var[1]
var

In [ ]:
Lsg = solve(gls[:-1], var)
Lsg

Wir hatten das beim ersten Mal mit $N=8$ gemacht.  Das sind zu wenige Daten.   Jetzt noch Mal mit $N=18$.

Aus Gründen, die ich nicht verstehe, muss man den Kernel zurücksetzen, bevor man mit dem neuen $N$ startet.

In [ ]:
#raise Unterbrechung

In [ ]:
Lsg[a[1]] = a[1]

In [ ]:
q = [Lsg[a[2*j+3]]/Lsg[a[2*j+1]] for j in range(int(N/2)-2)]
display(q)

In [ ]:
liste = []
for j in range(int(N/2-2)):
    m = Lsg[a[2*j+1]]/Lsg[a[2*j+3]]
    liste.append(m/(j+2))
liste

Also
$$ \frac{a_{2j+3}}{a_{2j+1}} = \frac1{(j+1)(j+2)} $$

Das bedeutet
$$ a_{2n+3} = \prod_{j=0}^n \frac1{(j+1)(j+2)} a_1 = \frac{a_1}{(n+1)!(n+2)!}. $$

Probe

In [ ]:
for j in range(int(N/2)-2):
    display((Lsg[a[2*j+1]], a[1]/factorial(j)/factorial(j+1)))

In [ ]:
S1 = Sum(x**(2*n+1)/factorial(n)/factorial(n+1), (n,0,oo))
S1

In [ ]:
u = S1.doit()
u

In [ ]:
srepr(u)

Eine zweite Lösung müsste man erhalten können, indem man einen Ansatz aus einer Potenzreihe und dem Produkt aus dem Logarithmus und einer Potenzreihe macht.

Das Reduktionsverfahren von d'Alembert führt auf ein schwieriges Integral.

In [ ]:
tmp = dgl.subs(y(x), u).doit()
tmp

http://dlmf.nist.gov

$$ I_{\nu-1}(z) - I_{\nu+1}(z) = \frac{2\nu}z I_\nu(z) $$

In [ ]:
(tmp.lhs - tmp.rhs).series(x, 0, 20)

## Pattern matching

In [ ]:
x = Symbol('x')

In [ ]:
x1 = Wild('x1')


In [ ]:
pattern = sin(x1)

In [ ]:
a = sin(2*x+5)

In [ ]:
m = a.match(pattern)
m

In [ ]:
b = 2*sin(x1/2)*cos(x1/2)
b

In [ ]:
b.subs(m)

In [ ]:
def expand_sin_x_halbe(term):
    x1 = Wild('x1')
    pattern = sin(x1)
    ersetzung = 2*sin(x1/2)*cos(x1/2)
    m = term.match(pattern)
    if m:
        return ersetzung.subs(m)
    else:
        return term    

In [ ]:
expand_sin_x_halbe(sin(x/2))

In [ ]:
series(expand_sin_x_halbe(sin(x)) - sin(x), x, 0, 20)


In [ ]:
a = 2*sin(x)
expand_sin_x_halbe(a)

In [ ]:
def expand_sin_x_halbe(ausdr):
    ausdr = S(ausdr)
    x1 = Wild('x1')
    pattern = sin(x1)
    ersetzung = 2*sin(x1/2)*cos(x1/2)
    m = ausdr.match(pattern)
    if m:
        res = ersetzung.subs(m)
    elif ausdr.is_Mul:
        res = 1
        for term in ausdr.args:
            res = res * expand_sin_x_halbe(term)
    elif ausdr.is_Add:
        res = 0
        for term in ausdr.args:
            res = res + expand_sin_x_halbe(term)
    else:
        res = ausdr
    return res

In [ ]:
expand_sin_x_halbe(sin(2*x))

In [ ]:
expand_sin_x_halbe(sin(2*x)/2)

In [ ]:
expand_sin_x_halbe(1+10*sin((x+1)**2))

In [ ]:
ausdr = (1 + sin(x/2))**3

In [ ]:
expand_sin_x_halbe(ausdr)

In [ ]:
ausdr.is_Pow

Mit etwas Fleiß bekommt man ein vollständiges Ersetzungssystem